# http://blog.8thandwalton.com/2014/06/supplier-glossary-fineline/

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from collections import Counter
from sklearn.externals import joblib

from sklearn.cross_validation import train_test_split
from sklearn.preprocessing import LabelEncoder

from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2

from sklearn.metrics import confusion_matrix

import xgboost
%matplotlib inline

In [2]:
train = pd.read_csv('./train.csv')
test = pd.read_csv('./test.csv')

In [3]:
train.shape

(647054, 7)

In [4]:
train.head()

,TripType,VisitNumber,Weekday,Upc,ScanCount,DepartmentDescription,FinelineNumber
0,999,5,Friday,68113152929,-1,FINANCIAL SERVICES,1000
1,30,7,Friday,60538815980,1,SHOES,8931
2,30,7,Friday,7410811099,1,PERSONAL CARE,4504
3,26,8,Friday,2238403510,2,PAINT AND ACCESSORIES,3565
4,26,8,Friday,2006613744,2,PAINT AND ACCESSORIES,1017


In [5]:
test.head()

,VisitNumber,Weekday,Upc,ScanCount,DepartmentDescription,FinelineNumber
0,1,Friday,72503389714,1,SHOES,3002
1,1,Friday,1707710732,1,DAIRY,1526
2,1,Friday,89470001026,1,DAIRY,1431
3,1,Friday,88491211470,1,GROCERY DRY GOODS,3555
4,2,Friday,2840015224,1,DSD GROCERY,4408


In [6]:
train_y = train.loc[:, ['VisitNumber', 'TripType']]
train_y.drop_duplicates('VisitNumber', inplace=True)
train_y.set_index('VisitNumber', inplace=True)

In [7]:
train_y.head() #This will end up being y labels 96000 trips

,TripType
VisitNumber,
5,999
7,30
8,26
9,8
10,8


In [8]:
test_y_cols = test.VisitNumber
test_y_cols.drop_duplicates(inplace = True) #This is a pandas series
#test_y_cols = test_y_cols.values

In [9]:
train.head()

,TripType,VisitNumber,Weekday,Upc,ScanCount,DepartmentDescription,FinelineNumber
0,999,5,Friday,68113152929,-1,FINANCIAL SERVICES,1000
1,30,7,Friday,60538815980,1,SHOES,8931
2,30,7,Friday,7410811099,1,PERSONAL CARE,4504
3,26,8,Friday,2238403510,2,PAINT AND ACCESSORIES,3565
4,26,8,Friday,2006613744,2,PAINT AND ACCESSORIES,1017


In [10]:
train_negatives = train[train.ScanCount < 0] #Can pivot into their own 'negative fineline counts'
train_negatives_agg = train_negatives.groupby(['VisitNumber']).agg({'ScanCount':np.sum}) #Negative Feature Count

In [11]:
train_negatives_agg.shape

(11077, 1)

In [12]:
train_uncategorized = train[pd.isnull(train.Upc)]
train_uncategorized_agg = train_uncategorized.groupby(['VisitNumber']).agg({'ScanCount':np.sum}) #Unknown Feature Count

In [13]:
train_uncategorized_agg.shape

(2754, 1)

In [14]:
test_negatives = test[test.ScanCount < 0]
test_negatives_agg = test.groupby(['VisitNumber']).agg({'ScanCount':np.sum}) #Negative Feature Count
test_uncategorized = test[pd.isnull(test.Upc)]
test_uncategorized_agg = test_uncategorized.groupby(['VisitNumber']).agg({'ScanCount':np.sum}) #Unknown Feature Count

In [15]:
train.drop(['TripType'], axis = 1, inplace = True)

In [16]:
train.Upc.fillna(-100, inplace=True)
train.DepartmentDescription.fillna('Unknown', inplace=True)
train.FinelineNumber.fillna(-100, inplace=True)

test.Upc.fillna(-100, inplace=True)
test.DepartmentDescription.fillna('Unknown', inplace=True)
test.FinelineNumber.fillna(-100, inplace=True)

In [17]:
train['FinelineNumber'] = train['FinelineNumber'].astype('int')
test['FinelineNumber'] = test['FinelineNumber'].astype('int')

In [18]:
train['DeptItems'] = train.DepartmentDescription +' ' + train.FinelineNumber.astype('str')
test['DeptItems'] = test.DepartmentDescription +' ' + test.FinelineNumber.astype('str')

In [19]:
full_df = pd.concat((train, test)) #Cannot Concant with ScanCount > 0.. some visit numbers will not be present

In [20]:
print full_df.shape
full_df.head()


(1300700, 7)


,VisitNumber,Weekday,Upc,ScanCount,DepartmentDescription,FinelineNumber,DeptItems
0,5,Friday,68113152929,-1,FINANCIAL SERVICES,1000,FINANCIAL SERVICES 1000
1,7,Friday,60538815980,1,SHOES,8931,SHOES 8931
2,7,Friday,7410811099,1,PERSONAL CARE,4504,PERSONAL CARE 4504
3,8,Friday,2238403510,2,PAINT AND ACCESSORIES,3565,PAINT AND ACCESSORIES 3565
4,8,Friday,2006613744,2,PAINT AND ACCESSORIES,1017,PAINT AND ACCESSORIES 1017


In [21]:
#full_df[full_df.VisitNumber==191319] #Useful to check

In [22]:
visit_days = full_df.loc[:,['VisitNumber','Weekday']]
visit_days.drop_duplicates('VisitNumber', inplace = True)
visit_days.set_index('VisitNumber', inplace = True)

In [23]:
visit_days.shape

(191348, 1)

In [24]:
#visit_days.set_index('VisitNumber', inplace = True)
#visit_days.sort_index(inplace = True)
#visit_days.reset_index(inplace = True)

In [25]:
#visit_days.sort(columns=['VisitNumber'], inplace=True)
#visit_days.head()

In [26]:
visit_days = pd.get_dummies(visit_days)
#visit_days.drop(['Weekday'], axis = 1, inplace = True)

In [27]:
visit_days.head()

,Weekday_Friday,Weekday_Monday,Weekday_Saturday,Weekday_Sunday,Weekday_Thursday,Weekday_Tuesday,Weekday_Wednesday
VisitNumber,,,,,,,
5,1,0,0,0,0,0,0
7,1,0,0,0,0,0,0
8,1,0,0,0,0,0,0
9,1,0,0,0,0,0,0
10,1,0,0,0,0,0,0


In [28]:
full_df.shape

(1300700, 7)

In [29]:
full_df.head()

,VisitNumber,Weekday,Upc,ScanCount,DepartmentDescription,FinelineNumber,DeptItems
0,5,Friday,68113152929,-1,FINANCIAL SERVICES,1000,FINANCIAL SERVICES 1000
1,7,Friday,60538815980,1,SHOES,8931,SHOES 8931
2,7,Friday,7410811099,1,PERSONAL CARE,4504,PERSONAL CARE 4504
3,8,Friday,2238403510,2,PAINT AND ACCESSORIES,3565,PAINT AND ACCESSORIES 3565
4,8,Friday,2006613744,2,PAINT AND ACCESSORIES,1017,PAINT AND ACCESSORIES 1017


In [30]:
#Should pivot with only positive scancounts so i can perform chisq
full_df_departments = pd.pivot_table(full_df[full_df.ScanCount>0], values='ScanCount', index='VisitNumber',columns='DeptItems', aggfunc=np.sum)
full_df_departments.fillna(0, inplace=True)

In [31]:
full_df_departments.head() #Will be incomplete because some ScanCounts **had** only negative values

DeptItems,1-HR PHOTO 110,1-HR PHOTO 120,1-HR PHOTO 130,1-HR PHOTO 141,1-HR PHOTO 150,1-HR PHOTO 160,1-HR PHOTO 1628,1-HR PHOTO 170,1-HR PHOTO 180,1-HR PHOTO 190,...,WIRELESS 870,WIRELESS 880,WIRELESS 890,WIRELESS 9,WIRELESS 940,WIRELESS 950,WIRELESS 965,WIRELESS 970,WIRELESS 990,WIRELESS 9998
VisitNumber,,,,,,,,,,,,,,,,,,,,,
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [32]:
full_df_departments['Totals'] = 0

In [33]:
for department in full_df_departments.columns[:-1]:
    full_df_departments['Totals'] += full_df_departments[department]

In [34]:
totals = full_df_departments[['Totals']]
full_df_departments.drop('Totals', axis=1, inplace = True)

In [35]:
train_long = train_y.join(full_df_departments)
train_long.fillna(0, inplace = True)

In [36]:
X_train = train_long.drop('TripType', axis = 1).values
y_train = train_long['TripType'].values

In [37]:
print X_train.shape, y_train.shape

(95674, 11048) (95674,)


In [38]:
chi_sq_best = SelectKBest(score_func=chi2, k = 7000)


In [39]:
chi_sq_best.fit(X_train,y_train)

SelectKBest(k=7000, score_func=<function chi2 at 0x7f6074c22398>)

In [40]:
np.sum(pd.isnull(chi_sq_best.pvalues_)) #646 null values in chisq not sure what this means

646

In [41]:
np.sum(chi_sq_best.pvalues_ < .00001) #Used 7000

7339

In [42]:
X_subset = chi_sq_best.transform(X_train) ##This takes into account Dept and Fineline number associations
X_subset.shape

(95674, 7000)

In [43]:
print train_y.head()
print train_uncategorized_agg.head()
print train_negatives_agg.head()
print totals.head()

             TripType
VisitNumber          
5                 999
7                  30
8                  26
9                   8
10                  8
             ScanCount
VisitNumber           
8                    1
259                  4
409                 -1
479                  1
484                 -4
             ScanCount
VisitNumber           
5                   -1
8                   -2
132                 -2
133                 -1
182                 -1
DeptItems    Totals
VisitNumber        
1                 4
2                 4
3                 1
4                 1
6                 1


In [44]:
train_negatives_agg.head()

,ScanCount
VisitNumber,
5,-1
8,-2
132,-2
133,-1
182,-1


In [45]:
X_additional_aggregates = train_y.join(train_uncategorized_agg,rsuffix='Uncategorized') #Should rename next time run
X_additional_aggregates = train_y.join(train_negatives_agg, rsuffix='Negatives')
X_additional_aggregates = train_y.join(visit_days, rsuffix='Days')
X_additional_aggregates = train_y.join(totals, rsuffix='Totals')

In [ ]:
X_additional_aggregates.drop('TripType', inplace = True)

In [50]:
X_additional_aggregates.fillna(0,inplace=True)

In [51]:
X_additional_aggregates.shape #Temporary 10 additional features, will probably want just by department agragates

(95674, 10)

In [53]:
for c in X_additional_aggregatesd

,ScanCount,ScanCountNegatives,Weekday_Friday,Weekday_Monday,Weekday_Saturday,Weekday_Sunday,Weekday_Thursday,Weekday_Tuesday,Weekday_Wednesday,Totals
VisitNumber,,,,,,,,,,
5,0,-1,1,0,0,0,0,0,0,0
7,0,0,1,0,0,0,0,0,0,2
8,1,-2,1,0,0,0,0,0,0,30
9,0,0,1,0,0,0,0,0,0,3
10,0,0,1,0,0,0,0,0,0,3
11,0,0,1,0,0,0,0,0,0,4
12,0,0,1,0,0,0,0,0,0,7
15,0,0,1,0,0,0,0,0,0,9
17,0,0,1,0,0,0,0,0,0,4


In [ ]:
full_df_departments.reset_index(inplace = True)

In [ ]:
departments_and_time = pd.merge(full_df_departments, visit_days, on='VisitNumber')

In [ ]:
days = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday','Saturday', 'Sunday']

for c in days:
    departments_and_time[c] = departments_and_time[c] * departments_and_time['Totals']

In [ ]:
departments_and_time.set_index('VisitNumber', inplace = True)

In [ ]:
X = departments_and_time.loc[train_y.VisitNumber,:].values
y = train_y[['TripType']].values

In [ ]:
print X.shape
print y.shape

In [ ]:
enc = LabelEncoder()
y = enc.fit_transform(y.ravel())

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X, y , test_size = 3000, random_state = 1)

In [ ]:
xgb = xgboost.XGBClassifier(max_depth = 13, n_estimators = 100,
                        objective='multi:softprob', subsample = .9, colsample_bytree=.8)

xgb.fit(X_train, y_train, eval_set = [(X_val, y_val)], eval_metric = 'mlogloss', early_stopping_rounds=5)


In [ ]:
X_test = departments_and_time.loc[test_y_cols,:].values

In [ ]:
y_probas = xgb.predict_proba(X_test)


In [ ]:
y_probas.shape

In [ ]:
#test = pd.read_csv('./test.csv')

In [ ]:
col_names = ['TripType_' + str(c) for c in enc.classes_]
submission = pd.DataFrame(np.round(y_probas, 3), index=test_y_cols, columns = col_names)

In [ ]:
submission.head()

In [ ]:
submission.reset_index(inplace = True)

submission.to_csv('Walmart_submission_XGB_Simple_DepartmentsAndTotal-1.csv', index=False)

In [ ]:
y_pred = xgb.predict(X_train)

In [ ]:
#cm = confusion_matrix(y_train,y_pred)
#plt.imshow(cm,cmap=plt.cm.Blues)

In [ ]:
#Generate confusion matrix to look at in excel

cm_df = pd.DataFrame(cm, index = enc.classes_, columns=enc.classes_)
cm_df.to_csv('Walmart_Confusion_Matrix.csv')